###           Classification of Cars using Feed Forward Network(Perceptron)

#### Data Preprocessing Part

In [16]:
#Importing the required Libraries
import os
import glob
import pickle
import random
import numpy as np
import cv2

In [17]:
#Getting the file from Different Directories
directories = [name for name in glob.glob('Dataset/class_*') if os.path.isdir(name)]
directories

['Dataset\\class_1',
 'Dataset\\class_2',
 'Dataset\\class_3',
 'Dataset\\class_4']

In [18]:
#Preparing the final target class used in the Algorithm
classes = []
for directory in directories:
    classes.append(directory.split('\\')[1])
classes

['class_1', 'class_2', 'class_3', 'class_4']

In [19]:
#The final feature dataset should be in the form of nested tuples,dictionaries,lists
#The Format specified is
#[(class,{actual_pathname:'absolute path to png file'})]

feature_data = []                      #The data used for training purpose
for directory in directories:
    paths = glob.glob(directory+'/*.jpg')
    current_class = directory.split('\\')[1]
    for path in paths:
        feature_dictionary = {}        #A dummy Dictionary used for converting it to tuples 
        feature_dictionary['path'] = path
        feature_data.append((current_class,feature_dictionary))

In [20]:
#We Extract data from Different Directories and then To feed it to the Neural Network Shuffle it 
#so than the Network is not biased and manual mappings cannot be done

random.shuffle(feature_data)

In [21]:
#Saving Data for fast access
with open('feature_data.pkl','wb') as f:
    pickle.dump(feature_data,f)
with open('classes.pkl','wb') as f:
    pickle.dump(classes,f)

#### Designing the Network

In [22]:
#Retrievng the Stored Data from the DB
with open('feature_data.pkl','rb') as f:
    feature_data = pickle.load(f)
with open('classes.pkl','rb') as f:
    classes = pickle.load(f)

### Training Algorithm Begins

In [23]:
#Step 1 Initialise Weights and Biases to small random values and set the Learning Rate

# Constants
bias = 1                        # Dummy Feature for use in setting constant factor in Training.
Train_Test_Ratio = 0.6           # Default Ratio of data to be used in Training vs. Testing.
Iterations = 20                 # Number of Training Iterations.(Epochs)
lr = 0.1                       # Set learning rate(Alpha)

In [24]:
#Defining Differernt Activation Functions Here Choice of Activation Functions?
def sigmoid(x):
    return (1/1+np.exp(-x))

def ReLu(x):
    return max(0,x)              

In [ ]:
#Defining the Perceptron Class

class MultiClassPerceptron:
    
    accuracy = 0
    
# A MultiClass Perceptron Model for training of Data,Predicting Analysing Features etc

    def __init__(self,classes,feature_data,hidden,train_test_ratio=Train_Test_Ratio,iterations=Iterations):
        self.classes = classes
        self.hidden = hidden
        self.feature_data = feature_data
        self.ratio = train_test_ratio
        self.iterations = iterations
        
        #Splitting Feature Data into Training and Testing Set For The Second step i;e training pair s:t
        random.shuffle(self.feature_data)
        self.train_set = self.feature_data[:int(len(self.feature_data)*self.ratio)]
        self.test_set = self.feature_data[int(len(self.feature_data)*self.ratio):]
        
        #Initialising the Weight Vectors in the Form of a Dictionary with BIAS terms between Input and Hidden Layers
        self.weight_vectors = {c:np.array([0.0 for _ in range(4901)]) for c in self.hidden}
        
        #Initialising the Weight Vectors in the Form of a Dictionary with BIAS terms between Hidden and Output Layers
        
        self.weight_hidd_out = {p:np.array([0.0 for _ in range(len(self.hidden))]) for p in self.classes}
        #Done with Step1 now train the network
        
    def train(self):

                
        """Train the Multi-Class Perceptron algorithm using the following method:

        Step2 : For each training pair s:t, the data (formatted as a feature vector) is read in, and the dot
                product is taken with each unique weight vector (which are all initially set to 0). 
                
        Step3 : Calculate the net I/P Yin = bias + summation(x*wts).
        
        Step4 : Pass it to the Activations Functions.
        
        Step5 : The class that
        yields the highest product is the class to which the data belongs. In the case this class is the
        correct value (matches with the actual category to which the data belongs), nothing happens, and the
        next data point is read in. However, in the case that the predicted value is wrong, the weight vectors a
        re corrected as follows: The feature vector is subtracted from the predicted weight vector, and added to
        the actual (correct) weight vector. This makes sense, as we want to reject the wrong answer, and accept
        the correct one."""
        #For Number of Epochs
        for _ in range(self.iterations):
        
            for category,feature_dictionary in self.train_set:

                #Obtaininig the Image and preprocessing it
                image = cv2.imread(feature_dictionary['path'])
                #Dimension Required for Resizing the Data
                dim = (70,70)

                resized_image = cv2.resize(image,dim,cv2.INTER_AREA)

                input_array = []

                #Normalising the Image between 0 and 1

                input_array = [np.mean(resized_image[i][j]/255) for i in range(resized_image.shape[0])
                              for j in range(resized_image.shape[1])]
                #Appending Extra Bias for each Input
                input_array.append(bias)
                input_vector = np.array(input_array)
                
                #Initialise ARG_MAX we perform our activations with that
                arg_max,predicted_class = 0,self.classes[0]
                
                #This is the Input for Hidden Layers
                input_array_hidden = []
                
                #We Make MultiClass Decision Rule Based on the above conditions
                for index in self.hidden:
                    current_activation = np.dot(input_vector,self.weight_vectors[index])
                    current_activation = ReLu(current_activation)
                    
                    input_array_hidden.append(current_activation)
                    #if current_activation >= arg_max:
                        #arg_max,predicted_class = current_activation,index
                
                for index in self.classes:
                    current_activation = np.dot(input_array_hidden,self.weight_hidd_out[index])
                    if current_activation >= arg_max:
                        arg_max,predicted_class = current_activation,index
                        
                #Updation of Weights And Biases
                #that is y!=t Update weights and biases
                #If y!=t we reduce its probability
                if not(category == predicted_class):
                    for index in self.hidden:
                        target = int(category[-1])
                        self.weight_vectors[int(index)] += [weight*lr*target for weight in input_vector]
#                         self.weight_vectors[predicted_class] -= [weight*lr for weight in input_vector]
                    
         
    def predict(self,feature_dictionary):
        
        #To check whether our model can generalise well as it has been trained so it has to learn well to Generalize well
        img = cv2.imread(feature_dictionary['path'])
        dim = (70,70)
        resized_image = cv2.resize(img,dim,cv2.INTER_AREA)
        
        input_array = []

        #Normalising the Image between 0 and 1

        input_array = [np.mean(resized_image[i][j]/255) for i in range(resized_image.shape[0])
                      for j in range(resized_image.shape[1])]
        
        
        input_array.append(bias)
        feature_vector = np.array(input_array)
        
        
        arg_max,predicted_class = 0,self.classes[0]
        
        #This is the Input for Hidden Layers
        input_array_hidden = []

        #We Make MultiClass Decision Rule Based on the above conditions
        for index in self.hidden:
            current_activation = np.dot(feature_vector,self.weight_vectors[index])
            current_activation = ReLu(current_activation)
                    
            input_array_hidden.append(current_activation)
                
        #We Make MultiClass Decision Rule Based on the above conditions
        for index in self.classes:
            current_activation = np.dot(input_array_hidden,self.weight_hidd_out[index])
            #current_activation = ReLu(current_activation)
            if current_activation >= arg_max:
                arg_max,predicted_class = current_activation,index
                
        return predicted_class
    
    def test_random_data(self):
        item = random.choice(self.test_set)
        print("Actual Class :" + item[0])
        pred_class = self.predict(item[1])
        print("Predicted Class :" + pred_class)
        
        
    def calculate_accuracy(self):
        
        """Calculates the accuracy of the classifier by running algorithm against test set and comparing
        the output to the actual categorization."""
        
        correct,incorrect = 0,0
        random.shuffle(self.feature_data)
        self.test_set = self.feature_data[int(len(self.feature_data)*self.ratio):]
        for feature_dictionary in self.test_set:
            actual_class = feature_dictionary[0]
            predicted_class = self.predict(feature_dictionary[1])
            
            if actual_class == predicted_class:
                correct += 1
            else:
                incorrect += 1
                
        print("ACCURACY")
        print("Accuracy of FF Model is ",(correct * 1.0) / ((correct + incorrect) * 1.0))
        
    
    def select_random_picture(self):
        #Reading Individual Image For Classification
        
        s = glob.glob(os.getcwd() + '\\Testing_Sample/*')
        
        #Rreading the Image Using Open CV
        img = cv2.imread(s[0])
        dim = (70,70)
        resized_image = cv2.resize(img,dim,cv2.INTER_AREA)
        
        input_array = []

        #Normalising the Image between 0 and 1

        input_array = [np.mean(resized_image[i][j]/255) for i in range(resized_image.shape[0])
                      for j in range(resized_image.shape[1])]
        
        
        input_array.append(bias)
        feature_vector = np.array(input_array)
        
        
        arg_max,predicted_class = 0,self.classes[0]
                
        #We Make MultiClass Decision Rule Based on the above conditions
        for index in self.classes:
            current_activation = np.dot(feature_vector,self.weight_vectors[index])
            if current_activation >= arg_max:
                arg_max,predicted_class = current_activation,index
                
        print("The Predicted Class for the Sample Image is")
        print(predicted_class)
            

In [ ]:
#The MAin function driver function
if __name__ == "__main__":
    hidden_neuron = [i for i in range(500)]
    classifier = MultiClassPerceptron(classes,feature_data,hidden_neuron)
    classifier.train()
    classifier.calculate_accuracy()
    #classifier.select_random_picture()